In [3]:
from Designer import SST, show

![Beam](2PL-beam.svg)

In [4]:
L1 = 3000.
L2 = 5500.
Pd = 30.
Pl = 60.

In [5]:
Fy = 345.
E = 200000.
G = 77000.
phi = 0.9

In [6]:
DeltaR = 360.

In [7]:
# Case 8
P = Pl*1000.
l = 2*L2 + L1
a = L2
DeltaI = P*a*(3*l*l-4*a*a)/(24.*E)
Ireq = DeltaI/(l/DeltaR)
show('P,l,a,*1E6,Ireq')

P    = 60000       
l    = 14000       
a    = 5500        
Ireq = 825.6 * 1E6 


In [8]:
Mf = (1.25*Pd + 1.5*Pl)*L2/1000.
Mf

701.25

In [9]:
# choose W610x113

In [19]:
from math import pi, sqrt

def check_Mr(Ix,Sx,Zx,Iy,J,Cw,D,B,T,W,Dsg,
             E=200000.,G=77000.,Fy=345.,L=0.,omega2=1.0,Ireq=0.,Mf=0.,Vf=0.,display=False):
    """This only works for W Shapes with unstiffened webs."""
    phi = 0.9
    if display:
        print('Section Properties and Dimensions:')
        show('Dsg,D,B,T,W,*1E6,Ix,*1E3,Sx,Zx,*1E6,Iy,*1E3,J,*1E9,Cw,*,E,G,Fy',data=locals())
        print('\nBeam Data:')
        show('*1E6,Ireq,*,Mf,L,omega2',data=locals())
    if Ix < Ireq:
        return False
    
    btlim = [x/sqrt(Fy) for x in [145,170,200]]
    hwlim = [x/sqrt(Fy) for x in [1100,1700,1900]]
    fclass = wclass = 4
    bt = B/(2*T)
    for i in range(1,4):
        if bt <= btlim[i-1]:
            fclass = i
            break
    hw = (D-2*T)/W
    for i in range(1,4):
        if hw <= hwlim[i-1]:
            wclass = i
            break
    sclass = max(fclass,wclass)
    
    if display:
        print('\nSection Class:')
        show('b/2t=bt,class 1=btlim[0],class 2=btlim[1],class 3=btlim[2],flange class=fclass',data=locals())
        show('h/w=hw,class 1=hwlim[0],class 2=hwlim[1],class 3=hwlim[2],web class=wclass,sect. class=sclass',data=locals())

    if sclass > 3:
        return False

    if sclass <= 2:
        Mpy = Mp = Fy*Zx * 1E-6
    else:
        Mpy = My = Fy*Sx * 1E-6
        
    if L > 0.:
        A = E*Iy*G*J
        B = Iy*Cw*((pi*E/L)**2)
        Mu = (omega2*pi/L)*sqrt(A+B) * 1E-6
        if Mu > 0.67*Mpy:
            Mr = min(1.15*phi*Mpy*(1-0.28*Mpy/Mu),phi*Mpy)
        else:
            Mr = phi*Mu
    else:
        Mr = phi*Mpy
        
    if display:
        if sclass < 3:
            s = 'Mp=Mpy,phi*Mp,Mu,0.67*Mp=0.67*Mpy,Mr'
        else:
            s = 'My=Mpy,phi*My,Mu,0.67*My=0.67*Mpy,Mr'
        print('\nMoment Strength:')
        show(s,data=locals())
        
    if Mr < Mf:
        return False
    
    hw = (D-2*T)/W
    Aw = D*W
    if hw <= 1014./sqrt(Fy):
        Fs = 0.66*Fy
    elif hw <= 1435./sqrt(Fy):
        Fs = 670*sqrt(Fy)/hw
    else:
        Fs = 961200./(hw**2)
    Vr = phi*Aw*Fs * 1E-3
    if display:
        print('\nShear Strength:')
        show('h/w=hw,1014/sqrt(Fy),1435/sqrt(Fy),Fs,Aw,Vr',data=locals())
    
    
    return dict(Mr=Mr,Vr=Vr)

In [11]:
wl = SST.section_tables(['W','WWF'],'Mass')
s = SST.select(wl,check_Mr,maxn=10,Mf=Mf,L=5500.,omega2=1.75,Ireq=Ireq)
s[['Mass','Avl','Use','Vr','Ix','Mr']]

,Mass,Avl,Use,Vr,Ix,Mr
W610x113,113.0,*C,3.0,1395.491328,8.750000e+08,905.921205
W610x125,125.0,*C,3.0,1492.464204,9.850000e+08,1027.691326
W690x125,125.0,*,3.0,1610.643098,1.190000e+09,1138.657116
W760x134,134.0,*,3.0,1654.448129,1.500000e+09,1318.039848
W530x138,138.0,*C,3.0,1653.846579,8.610000e+08,1022.119112
W610x140,140.0,*C,3.0,1656.387711,1.120000e+09,1179.469541
W690x140,140.0,*,3.0,1738.134288,1.360000e+09,1311.080448
W760x147,147.0,*,3.0,2036.922228,1.660000e+09,1461.030129
W530x150,150.0,*,3.0,1413.217773,1.010000e+09,1288.575000
W690x152,152.0,*,3.0,1846.993104,1.510000e+09,1454.577155


In [12]:
s = SST.select(wl,check_Mr,maxn=10,Mf=Mf,L=5500.,omega2=1.75,Ireq=0)
s[['Mass','Avl','Use','Ix','Mr']]

,Mass,Avl,Use,Ix,Mr
W530x101,101.0,*C,3.0,6.170000e+08,705.626655
W610x101,101.0,*C,3.0,7.640000e+08,784.493653
W530x109,109.0,*C,3.0,6.670000e+08,771.993036
W460x113,113.0,*,3.0,5.560000e+08,826.399680
W610x113,113.0,*C,3.0,8.750000e+08,905.921205
W410x114,114.0,*,3.0,4.680000e+08,768.390905
W360x122,122.0,*,2.0,3.650000e+08,704.835000
W530x123,123.0,*C,3.0,7.610000e+08,891.968931
W610x125,125.0,*C,3.0,9.850000e+08,1027.691326
W690x125,125.0,*,3.0,1.190000e+09,1138.657116


In [13]:
s = SST.select(wl,check_Mr,maxn=10,Mf=Mf,L=4000.,omega2=1.,Ireq=Ireq) 
s[['Mass','Avl','Use','Ix','Mr']]

,Mass,Avl,Use,Ix,Mr
W610x113,113.0,*C,3.0,8.750000e+08,905.737803
W610x125,125.0,*C,3.0,9.850000e+08,1024.452359
W690x125,125.0,*,3.0,1.190000e+09,1144.921031
W760x134,134.0,*,3.0,1.500000e+09,1329.084463
W530x138,138.0,*C,3.0,8.610000e+08,1004.268427
W610x140,140.0,*C,3.0,1.120000e+09,1171.882014
W690x140,140.0,*,3.0,1.360000e+09,1315.122755
W760x147,147.0,*,3.0,1.660000e+09,1470.495000
W530x150,150.0,*,3.0,1.010000e+09,1288.575000
W690x152,152.0,*,3.0,1.510000e+09,1456.032599


In [14]:
SST.props.loc['Avl'].Desc

Availability, '*' indicates a section not produced in Canada. '+' indicates availability should be checked with supplier.

In [20]:
SST.get('W610x113').call(check_Mr,L=5500,omega2=1.75,display=True)

Section Properties and Dimensions:
Dsg = W610x113   
D   = 608        
B   = 228        
T   = 17.3       
W   = 11.2       
Ix  = 875 * 1E6  
Sx  = 2880 * 1E3 
Zx  = 3290 * 1E3 
Iy  = 34.3 * 1E6 
J   = 1120 * 1E3 
Cw  = 2990 * 1E9 
E   = 200000     
G   = 77000      
Fy  = 345        

Beam Data:
Ireq   = 0 * 1E6 
Mf     = 0       
L      = 5500    
omega2 = 1.75    

Section Class:
b/2t         = 6.59  
class 1      = 7.807 
class 2      = 9.152 
class 3      = 10.77 
flange class = 1     
h/w         = 51.2  
class 1     = 59.22 
class 2     = 91.52 
class 3     = 102.3 
web class   = 1     
sect. class = 1     

Moment Strength:
Mp      = 1135  
phi*Mp  = 1022  
Mu      = 1389  
0.67*Mp = 760.5 
Mr      = 905.9 

Shear Strength:
h/w           = 51.2  
1014/sqrt(Fy) = 54.59 
1435/sqrt(Fy) = 77.26 
Fs            = 227.7 
Aw            = 6810  
Vr            = 1395  


{Mr: 905.9212054870513, Vr: 1395.491328}

In [21]:
SST.get('W610x113').call(check_Mr,L=3000,omega2=1,display=True)

Section Properties and Dimensions:
Dsg = W610x113   
D   = 608        
B   = 228        
T   = 17.3       
W   = 11.2       
Ix  = 875 * 1E6  
Sx  = 2880 * 1E3 
Zx  = 3290 * 1E3 
Iy  = 34.3 * 1E6 
J   = 1120 * 1E3 
Cw  = 2990 * 1E9 
E   = 200000     
G   = 77000      
Fy  = 345        

Beam Data:
Ireq   = 0 * 1E6 
Mf     = 0       
L      = 3000    
omega2 = 1       

Section Class:
b/2t         = 6.59  
class 1      = 7.807 
class 2      = 9.152 
class 3      = 10.77 
flange class = 1     
h/w         = 51.2  
class 1     = 59.22 
class 2     = 91.52 
class 3     = 102.3 
web class   = 1     
sect. class = 1     

Moment Strength:
Mp      = 1135  
phi*Mp  = 1022  
Mu      = 2363  
0.67*Mp = 760.5 
Mr      = 1017  

Shear Strength:
h/w           = 51.2  
1014/sqrt(Fy) = 54.59 
1435/sqrt(Fy) = 77.26 
Fs            = 227.7 
Aw            = 6810  
Vr            = 1395  


{Mr: 1016.7503035611546, Vr: 1395.491328}